Pixel-Perfect SfM Demo file

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path

from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

# Setup

In [ ]:
images = Path('datasets/fountain')
outputs = Path('outputs/fountain3/')
!rm -rf $outputs
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
raw_dir = outputs / "raw"
ref_dir = outputs / "ref"

In [ ]:
feature_conf = extract_features.confs['superpoint_aachen']
#matcher_conf = match_features.confs['superglue']
matcher_conf = match_features.confs['superpoint+lightglue']

# 3D mapping and refinement

In [ ]:
references = [str(p.relative_to(images)) for p in (images / 'mapping/').iterdir()]
print(len(references), "mapping images")
plot_images([read_image(images / r) for r in references[:4]], dpi=50)

Then we extract features and match them across image pairs. Since we deal with few images, we simply match all pairs exhaustively.

In [ ]:
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

Now we run the reconstruction with and without the featuremetric refinement. For this dataset, when computing the dense features, we resize the images such that they are not larger than 1024 pixels.

In [ ]:
# run pixsfm
sfm = PixSfM({"dense_features": {"max_edge": 1024}})
refined, sfm_outputs = sfm.reconstruction(ref_dir, images, sfm_pairs, features, matches, image_list=references)
# here ref is pycolmap.Reconstruction object

# run the raw geometric SfM for comparison
raw_sfm = PixSfM({"KA":{"apply": False}, "BA": {"apply": False}})
raw, _ = raw_sfm.reconstruction(raw_dir, images, sfm_pairs, features, matches, image_list=references)

In [ ]:
print("Raw", raw.summary())
print("Refined", refined.summary())

# Visualization
To visualize both models together, we rigidly transform the raw reconstruction so that the 3D points common to the two 3D models are aligned.

In [ ]:
raw.align_points(refined, max_error=0.005, min_inlier_ratio=0.9, min_overlap=3)

In [ ]:
fig3d = init_figure()
args = dict(max_reproj_error=3.0, min_track_length=2, cs=1)
plot_reconstruction(fig3d, raw, color='rgba(255, 0, 0, 0.5)', name="raw", **args)
plot_reconstruction(fig3d, refined, color='rgba(0, 255, 0, 0.5)', name="refined", **args)
fig3d.show()

In [ ]:
img = refined.images[refined.reg_image_ids()[0]]
cam = refined.cameras[img.camera_id]
fig = init_image(images / img.name)    
plot_points2D(fig, [p2D.xy for p2D in img.points2D if p2D.has_point3D()])
plot_points2D(fig, cam.world_to_image(img.project(refined)), color='rgba(255, 0, 0, 0.5)')
fig.show()

# Export

In [ ]:
from hloc.utils.read_write_model import read_model, write_points3D_text

model_dir = ref_dir
output_path = model_dir / 'output_points_ref.ply'

cameras, images, points3D = read_model(path= ref_dir)

pt_data = []
for point_id, point in points3D.items():
    x, y, z = point.xyz
    r, g, b = point.rgb
    pt_data.append(f"{x} {y} {z} {r} {g} {b}")

# Write the PLY file
with open(output_path, 'w') as ply_file:
    ply_file.write(f"ply\nformat ascii 1.0\nelement vertex {len(points3D)}\n")
    ply_file.write("property float x\nproperty float y\nproperty float z\n")
    ply_file.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
    ply_file.write("end_header\n")
    ply_file.write('\n'.join(pt_data))

print(f"Exported {len(points3D)} 3D points to {output_path}")